This is a Python code version of the original location-guard code in Java (https://github.com/chatziko/location-guard#location-guard)

The Geo-Indistinguishability notion can be consulted in the original paper: Geo-Indistinguishability: Differential Privacy for Location-Based Systems (https://doi.org/10.1145/2508859.2516735). 

Arxived version: https://arxiv.org/abs/1212.1984

## Geo-Indistinguishability

In [14]:
#For the original code, please refer to: https://github.com/chatziko/location-guard/blob/master/src/js/common/laplace.js

import numpy as np
from mpmath import * #to import the Lambert W function (http://en.wikipedia.org/wiki/Lambert_W_function)

earth_radius = 6378137

# Converting Cartesian Coordinates (x,y) to degrees
def getLatLon(x, y): 
    long_in_rad = x/earth_radius 
    lat_in_rad = 2*np.atan(np.exp(y/earth_radius))-(np.pi/2)
    latitude = np.degrees(lat_in_rad)
    longitude = np.degrees(long_in_rad)
    return latitude, longitude

# Convert degrees (latitude, longitude) to Cartesian
def getCartesian(latitude, longitude): # ok
    x = earth_radius*np.radians(longitude)
    y = earth_radius*np.log(np.tan(np.pi/4+np.radians(latitude)/2))
    return x, y             

#This is the inverse cumulative polar laplacian distribution function. 
def inverseCumulativeGamma(epsilon, z): 
    x = (z-1)/np.e
    return -(lambertw(x, k = -1)+1)/epsilon 

# http://movable-type.co.uk/scripts/latlong.html
def addVectorToPos(latitude, longitude, distance, angle):
    ang_distance = distance/earth_radius
    #lat1, lon1 = getCartesian(latitude, longitude)
    lat1 = np.radians(latitude)
    lon1 = np.radians(longitude)
    lat2 = np.arcsin(float(np.sin(float(lat1))*np.cos(float(ang_distance))+np.cos(float(lat1))*np.sin(float(ang_distance))*np.cos(float(angle))))
    lon2 = lon1 + np.arctan2(
        np.sin(float(angle))*np.sin(float(ang_distance))*np.cos(float(lat1)),
        np.cos(float(ang_distance))-np.sin(float(lat1))*np.sin(float(lat2)))
    lon2 = (lon2 + 3*np.pi)%(2*np.pi) - np.pi #normalization to -180,...,180
    return np.degrees(lat2), np.degrees(lon2)#, distance, angle

# Returns destination position given distance and angle from start position
def addNoise(epsilon, latitude, longitude):
    theta = np.random.random()*np.pi*2
    z = np.random.random()
    r = inverseCumulativeGamma(epsilon, z)
    return addVectorToPos(latitude, longitude, r, theta)

## Usage example

In [16]:
#for reproducibility
np.random.seed(0)

#coordinates of São Paulo city in brazil
latitude, longitude = -23.554570187422183, -46.64289055544784

#level of privacy
l = 1

#radius (in meters)
radius = 200

#privacy level
epsilon = l/radius
addNoise(epsilon, latitude, longitude)

(-23.558872984176098, -46.644377150295846)